<a href="https://colab.research.google.com/github/abdala9512/fake-news-poc/blob/main/pryectoGrado_FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis exploratorio 1

#### Librerias

In [77]:
# NLKT
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [124]:
# Modulos nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import PlaintextCorpusReader
from nltk.probability import FreqDist
from nltk.corpus import PlaintextCorpusReader
from nltk.probability import ConditionalFreqDist
# Otras librerias
import pandas as pd
import string
import re
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import chardet
import unidecode

In [79]:
# Stopwords
stop_words = set(stopwords.words(['spanish','english']))

# Importar datos

In [125]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
# Rutas
root_true = '/content/drive/MyDrive/ProyectoGrado/Datos/data_fake_news/Veridicas'
root_fake = '/content/drive/MyDrive/ProyectoGrado/Datos/data_fake_news/Falsas'

In [127]:
# Noticias falsas
fake_list = PlaintextCorpusReader(root_fake, '.*')
fake_news =fake_list.fileids()

# Carga de archivos
fake_docs_raw = []

for file in fake_news:
    with open(root_fake+'/'+file, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    
    try:
        fake_list = PlaintextCorpusReader(root_fake, file, encoding=encoding)
        fake_docs_raw.append(fake_list.raw(file))
    except UnicodeDecodeError:
        print(f"Error: unable to decode {file} with encoding {encoding}")

Error: unable to decode F- EL DIÓXIDO DE CLORO NO CURA LA COVID-19 COMO RECOMIENDA LA MODELO COLOMBIANA NATALIA PARÍS.txt with encoding Windows-1254
Error: unable to decode F- Escándalo por comercialización de vacuna Pfizer contra COVID sin estudio de contagio es desinformación..txt with encoding Windows-1254
Error: unable to decode F- Mensaje desinforma sobre restricciones en varios países por cuarta ola de COVID-19.txt with encoding Windows-1254
Error: unable to decode F-No, este pantallazo no prueba que Tribunal de Colombia ordenó suspender el plan de vacunación.txt with encoding Windows-1254


In [128]:
# Noticias veridicas
true_list = PlaintextCorpusReader(root_true, '.*')
true_news =true_list.fileids()

# Carga de archivos
true_docs_raw = []

for file in true_news:
    with open(root_true+'/'+file, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    
    try:
        true_list = PlaintextCorpusReader(root_true, file, encoding=encoding)
        true_docs_raw.append(true_list.raw(file))
    except UnicodeDecodeError:
        print(f"Error: unable to decode {file} with encoding {encoding}")

# Funciones

In [129]:
def lexical_diversity(text):
    """
    Calcula la diversidad lexica de un texto.
    numero palabras unicas / numero total de palabras
    """
    return len(set(text)) / len(text)

In [130]:
def summary(text, name, k):
    '''
    Resumen descriptivo de cada noticia.

    text: texto a analizar
    name: nombre del texto
    k: longitud para identificar palabras raras
    '''
    num_sent = len(text.sents(name)) #Conteo de oraciones
    num_tkns = len(text.words(name)[:]) #Conteo de palabras

    #Conteo de palabras vacías
    stopwords = nltk.corpus.stopwords.words(['spanish','english'])
    num_empty = len([w for w in text.words(name)[:] if w.lower() in stopwords])

    #Número medio de oraciones por comentario
    if len(text.paras(name)) > 0:
        mean_sent = sum(len(p) for p in text.paras(name)) / float(len(text.paras(name)))
        #Número medio de tokens por oración
        mean_tkns = sum(len(s) for s in text.sents(name)) / float(len(text.sents(name)))
    else: 
        mean_sent = 0
        mean_tkns = 0

    #Palabras comunes y largas
    content = [word.lower() for word in text.words(name)[:] if word.lower() not in stopwords and word.isalpha()]
    fdist1 = FreqDist(content)
    long_words = [w for w in fdist1 if len(w) >= k]

    return("-"*100,
            "Nombre: " + name,
            "Oraciones en la noticia: " + str(num_sent),
            "Tokens en la noticia: " + str(num_tkns),
            "Palabras vacias en la noticia: " + str(num_empty),
            "Número medio de oraciones en la noticia: " + str(mean_sent),
            "Número medio de tokens por oración: " + str(mean_tkns),
            "Total palabras (no vacías, sin signos de puntuación): " + str(len(content)),
            "Top 10 palabras más comunes (sin tener en cuenta las palabras vacías): " + str(fdist1.most_common(10)),
            "Diversidad léxica: " + str(len(content)/num_tkns if num_tkns > 0 else 0),
            "Total palabras con una única aparición: " + str(len(fdist1.hapaxes())),
            "Palabras más largas (>=:" + str(k) + "): " + str(long_words))

# Resumen

#### Fakenews

In [131]:
[summary(text = fake_list,name = name, k = 13 ) for name in fake_news]

[('----------------------------------------------------------------------------------------------------',
  'Nombre: F - Hacer gargaras con agua y sal elimina el coronavirus.txt',
  'Oraciones en la noticia: 24',
  'Tokens en la noticia: 749',
  'Palabras vacias en la noticia: 299',
  'Número medio de oraciones en la noticia: 1.5',
  'Número medio de tokens por oración: 31.208333333333332',
  'Total palabras (no vacías, sin signos de puntuación): 339',
  "Top 10 palabras más comunes (sin tener en cuenta las palabras vacías): [('agua', 12), ('coronavirus', 11), ('gárgaras', 6), ('caliente', 5), ('nuevo', 5), ('garganta', 5), ('sal', 4), ('pulmones', 4), ('puede', 4), ('recomendaciones', 4)]",
  'Diversidad léxica: 0.45260347129506007',
  'Total palabras con una única aparición: 187',
  "Palabras más largas (>=:13): ['recomendaciones', 'publicaciones', 'tradicionales', 'respiratorias', 'inmediatamente']"),
 ('-------------------------------------------------------------------------------

#### Truenews

In [132]:
[summary(text = true_list,name = name, k = 13 ) for name in true_news]

[('----------------------------------------------------------------------------------------------------',
  'Nombre: V - Así fue como Colombia dejó de producir vacunas (1).txt',
  'Oraciones en la noticia: 73',
  'Tokens en la noticia: 1873',
  'Palabras vacias en la noticia: 812',
  'Número medio de oraciones en la noticia: 2.3548387096774195',
  'Número medio de tokens por oración: 25.65753424657534',
  'Total palabras (no vacías, sin signos de puntuación): 821',
  "Top 10 palabras más comunes (sin tener en cuenta las palabras vacías): [('vacuna', 16), ('producción', 14), ('vacunas', 12), ('nacional', 10), ('instituto', 8), ('covid', 6), ('ins', 6), ('años', 6), ('colombia', 5), ('salud', 5)]",
  'Diversidad léxica: 0.4383342231713828',
  'Total palabras con una única aparición: 410',
  "Palabras más largas (>=:13): ['internacional', 'estabilización', 'conversaciones', 'recientemente', 'circunstancias', 'infortunadamente', 'antiamarílica', 'rehidratación', 'reestructuraciones', 'me

# Procesamiento

In [ ]:
docs_raw = []
for i in range(21):
  docs_raw.append(lista.raw(fake_news[i]))

In [88]:
# Stemmer
ps = PorterStemmer()